<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction-to-Cross-Validation---Lab" data-toc-modified-id="Introduction-to-Cross-Validation---Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction to Cross-Validation - Lab</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Let's-get-started" data-toc-modified-id="Let's-get-started-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Let's get started</a></span><ul class="toc-item"><li><span><a href="#Train-test-split" data-toc-modified-id="Train-test-split-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Train-test split</a></span></li><li><span><a href="#Fit-the-model" data-toc-modified-id="Fit-the-model-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Fit the model</a></span></li><li><span><a href="#Residuals-and-MSE" data-toc-modified-id="Residuals-and-MSE-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Residuals and MSE</a></span></li></ul></li><li><span><a href="#Cross-Validation:-let's-build-it-from-scratch!" data-toc-modified-id="Cross-Validation:-let's-build-it-from-scratch!-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Cross-Validation: let's build it from scratch!</a></span><ul class="toc-item"><li><span><a href="#Create-a-cross-validation-function" data-toc-modified-id="Create-a-cross-validation-function-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Create a cross-validation function</a></span></li><li><span><a href="#Apply-it-to-the-Ames-Housing-data" data-toc-modified-id="Apply-it-to-the-Ames-Housing-data-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Apply it to the Ames Housing data</a></span></li><li><span><a href="#Perform-a-linear-regression-for-each-fold-and-calculate-the-training-and-test-error" data-toc-modified-id="Perform-a-linear-regression-for-each-fold-and-calculate-the-training-and-test-error-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Perform a linear regression for each fold and calculate the training and test error</a></span></li></ul></li><li><span><a href="#Cross-Validation-using-Scikit-Learn" data-toc-modified-id="Cross-Validation-using-Scikit-Learn-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Cross-Validation using Scikit-Learn</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Summary</a></span></li></ul></li></ul></div>

# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

### Train-test split

Perform a train-test split with a test set of 20%.

In [2]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split


In [3]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


In [4]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))


1168 292 1168 292


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [5]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

linreg.fit(X_train, y_train)
y_hat_test = linreg.predict(X_test)


### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [6]:
from sklearn.metrics import mean_squared_error
test_residuals = y_hat_test - y_test

test_mse = mean_squared_error(y_test, y_hat_test)
test_mse


0.16796226631640335

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [7]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    data = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations//k
    leftovers = num_observations%k
    folds = []
    start_obs = 0
    for fold_n in range(1,k+1):
        if fold_n <= leftovers:
            #Fold Size will be 1 larger to account for leftovers
            fold =  data.iloc[start_obs : start_obs+fold_size+1] 
            folds.append(fold)
            start_obs +=  fold_size + 1
        else:
            fold =  data.iloc[start_obs : start_obs+fold_size] 
            folds.append(fold)
            start_obs +=  fold_size
            
    return folds 


### Apply it to the Ames Housing data

In [8]:
# Make sure to concatenate the data again
ames_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [9]:
# Apply kfolds() to ames_data with 5 folds
ames_folds = kfolds(ames_data, 5)

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [10]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(ames_folds) if i!=n])
    test = ames_folds[n]
    # Fit a linear regression model
    linreg.fit(X_train, y_train)
    #Evaluate Train and Test Errors
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    train_residuals = y_hat_train - y_train
    test_residuals = y_hat_test - y_test
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))
print(train_errs)
print(test_errs)


[0.16235877826050601, 0.16235877826050601, 0.16235877826050601, 0.16235877826050601, 0.16235877826050601]
[0.16796226631640335, 0.16796226631640335, 0.16796226631640335, 0.16796226631640335, 0.16796226631640335]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [11]:
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score

mse = make_scorer(mean_squared_error)

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring=mse)


Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [12]:
cv_5_results.mean()

0.17702834210001123

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 